In [5]:
%load_ext Cython

In [ ]:
!pip install line_profiler

In [8]:
%load_ext line_profiler

In [101]:
# coding = 'utf-8'
import numpy as np
import pandas as pd

y = np.random.randint(2, size=(1000000, 1))
x = np.random.randint(10, size=(1000000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

# target_mean_v1

In [ ]:
def target_mean_v1(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  for i in range(data.shape[0]):
      groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
      result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
  return result

In [ ]:
%lprun -f target_mean_v1 target_mean_v1(data, 'y', 'x')

In [ ]:

res = target_mean_v1(data, 'y', 'x')
print(res)

[0.53909465 0.51709402 0.50790514 ... 0.49083503 0.49083503 0.53585657]


# target_mean_v2

优化算法，把代码复杂度到从 O(n2) 降到 O(n)



In [ ]:
def target_mean_v2(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  val_dict = dict()
  cnt_dict = dict()
  for i in range(data.shape[0]):
    x_key = data.loc[i, x_name]
    y_val = data.loc[i, y_name]
    if x_key not in val_dict.keys():
      val_dict[x_key] = y_val
      cnt_dict[x_key] = 1
    else:
      val_dict[x_key] += y_val
      cnt_dict[x_key] += 1
  for i in range(data.shape[0]):
    x_key = data.loc[i, x_name]
    y_val = data.loc[i, y_name]
    result[i] = (val_dict[x_key] - y_val) / (cnt_dict[x_key] - 1)
  return result

In [ ]:
%lprun -f target_mean_v2 target_mean_v2(data, 'y', 'x')

In [ ]:
%%timeit
res = target_mean_v2(data, 'y', 'x')

10 loops, best of 3: 154 ms per loop


# target_mean_v3

经过上一轮测试，我们发现在代码的`Hotspots`在了`loc`方法上。因为本纯小白对`DataFrame`的取数操作不了解，所以我决定测试一下几种取数操作的性能如何。

测试结果如下：
at > loc >> iat > iloc

其中，at 的测试结果最优。

In [ ]:
def target_mean_v3(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  val_dict = dict()
  cnt_dict = dict()
  for i in range(data.shape[0]):
    x_key = data.loc[i, x_name]
    y_val = data.iloc[i, 0]
    if x_key not in val_dict.keys():
      val_dict[x_key] = y_val
      cnt_dict[x_key] = 1
    else:
      val_dict[x_key] += y_val
      cnt_dict[x_key] += 1
  for i in range(data.shape[0]):
    x_key = data.at[i, x_name]
    y_val = data.iat[i, 0]
    result[i] = (val_dict[x_key] - y_val) / (cnt_dict[x_key] - 1)
  return result

In [ ]:
%lprun -f target_mean_v3 target_mean_v3(data, 'y', 'x')

In [ ]:
%%timeit
res = target_mean_v3(data, 'y', 'x')

1 loop, best of 3: 293 ms per loop


# target_mean_v4

经过上一轮测试，我们发现 at 最优，但是它依然是这段代码的 Hotspots，因此，本轮优化，我换一种思路，直接把 DataFrame 操作移动到循环外，循环中使用 list 索引取值

In [ ]:
def target_mean_v4(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  val_dict = dict()
  cnt_dict = dict()
  x_values = data[x_name].values
  y_values = data[y_name].values
  for i in range(data.shape[0]):
    x_key = x_values[i]
    y_val = y_values[i]
    if x_key not in val_dict.keys():
      val_dict[x_key] = y_val
      cnt_dict[x_key] = 1
    else:
      val_dict[x_key] += y_val
      cnt_dict[x_key] += 1
  for i in range(data.shape[0]):
    x_key = x_values[i]
    y_val = y_values[i]
    result[i] = (val_dict[x_key] - y_val) / (cnt_dict[x_key] - 1)
  return result

In [ ]:
%lprun -f target_mean_v4 target_mean_v4(data, 'y', 'x')

In [ ]:
%%timeit
res = target_mean_v4(data, 'y', 'x')

100 loops, best of 3: 8.85 ms per loop


# target_mean_v5（负优化）

把 DataFrame 的 x, y 分别弄成 array 之后，速度飙升，那么如果使用多重赋值，结果会怎么样？

结论：这一步，进行了一次负优化

In [ ]:
def target_mean_v5(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  val_dict = dict()
  cnt_dict = dict()
  x_values = data[x_name].values
  y_values = data[y_name].values
  for i in range(data.shape[0]):
    y_val, x_key = y_values[i], x_values[i]
    if x_key not in val_dict.keys():
      val_dict[x_key] = y_val
      cnt_dict[x_key] = 1
    else:
      val_dict[x_key] += y_val
      cnt_dict[x_key] += 1
  for i in range(data.shape[0]):
    y_val, x_key = y_values[i], x_values[i]
    result[i] = (val_dict[x_key] - y_val) / (cnt_dict[x_key] - 1)
  return result

In [ ]:
%lprun -f target_mean_v5 target_mean_v5(data, 'y', 'x')

In [ ]:
%%timeit
res = target_mean_v5(data, 'y', 'x')

100 loops, best of 3: 8.77 ms per loop


# target_mean_v6

经过 v4 的优化之后，已经达到了我所能优化的 python 代码的顶峰了，这一轮的 v6，再把 `data.shape[0]` 提成局部变量，减少索引。然后 v6 就作为 python 代码优化的最终版本了！接下来的 cython 将在 v6 的基础上继续优化！

In [ ]:
def target_mean_v6(data, y_name, x_name):
  data_shape = data.shape[0]
  result = np.zeros(data_shape)
  val_dict = dict()
  cnt_dict = dict()
  x_values = data[x_name].values
  y_values = data[y_name].values
  for i in range(data_shape):
    x_key = x_values[i]
    y_val = y_values[i]
    if x_key not in val_dict.keys():
      val_dict[x_key] = y_val
      cnt_dict[x_key] = 1
    else:
      val_dict[x_key] += y_val
      cnt_dict[x_key] += 1
  for i in range(data_shape):
    x_key = x_values[i]
    y_val = y_values[i]
    result[i] = (val_dict[x_key] - y_val) / (cnt_dict[x_key] - 1)
  return result

In [ ]:
%lprun -f target_mean_v6 target_mean_v6(data, 'y', 'x')

In [ ]:
%%timeit
res = target_mean_v6(data, 'y', 'x')

100 loops, best of 3: 8.57 ms per loop


# target_mean_v7

代码改成 cython，加上类型

本轮优化到了 1ms

In [46]:
%%cython
import numpy as np

cpdef target_mean_v7(data, y_name, x_name):
  cdef:
    int data_shape = data.shape[0]
    double[:] result = np.zeros(data_shape, dtype=np.float64)
    dict val_dict = dict()
    dict cnt_dict = dict()
    long[:] x_values = data[x_name].values
    long[:] y_values = data[y_name].values
  for i in range(data_shape):
    x_key = x_values[i]
    y_val = y_values[i]
    if x_key not in val_dict.keys():
      val_dict[x_key] = y_val
      cnt_dict[x_key] = 1
    else:
      val_dict[x_key] += y_val
      cnt_dict[x_key] += 1
  for i in range(data_shape):
    x_key = x_values[i]
    y_val = y_values[i]
    result[i] = (val_dict[x_key] - y_val) / (cnt_dict[x_key] - 1)
  return result

In [47]:
%%timeit
res = target_mean_v7(data, 'y', 'x')

1000 loops, best of 3: 1.18 ms per loop


# target_mean_v8

再cython中把dict去掉（因为不能指定类型），同时优化一下算法（因为 x 的值是 0-10 是确定的，所以可以将dict改成数组，数组下标就作为key了）




In [89]:
%%cython
import numpy as np

cpdef target_mean_v8(data, y_name, x_name):
  cdef:
    int data_shape = data.shape[0]
    double[:] result = np.zeros(data_shape, dtype=np.float64)
    long[:] val_dict = np.zeros(10, dtype=np.int64)
    double[:] cnt_dict = np.zeros(10, dtype=np.float64)
    long[:] x_values = data[x_name].values
    long[:] y_values = data[y_name].values
  for i in range(data_shape):
    val_dict[x_values[i]] += y_values[i]
    cnt_dict[x_values[i]] += 1

  for i in range(data_shape):
    result[i] = (val_dict[x_values[i]] - y_values[i]) / (cnt_dict[x_values[i]] - 1)
  return result

In [90]:
%%timeit
res = target_mean_v8(data, 'y', 'x')

The slowest run took 12.21 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 51.7 µs per loop


# target_mean_v9（目前的最优情况，100万 7ms）

加上

```
@cython.boundscheck(False)
@cython.wraparound(False)
```

防止数组越界，同时把array变成裸数组？

加上之后，速度相比 v8 又提升了将近一倍，达到了 34.4 us

In [102]:
%%cython
cimport cython
import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v9(data, y_name, x_name):
  cdef:
    int data_shape = data.shape[0]
    double[:] result = np.zeros(data_shape, dtype=np.float64)
    long[:] val_dict = np.zeros(10, dtype=np.int64)
    double[:] cnt_dict = np.zeros(10, dtype=np.float64)
    long[:] x_values = data[x_name].values
    long[:] y_values = data[y_name].values
  for i in range(data_shape):
    val_dict[x_values[i]] += y_values[i]
    cnt_dict[x_values[i]] += 1

  for i in range(data_shape):
    result[i] = (val_dict[x_values[i]] - y_values[i]) / (cnt_dict[x_values[i]] - 1)
  return result

In [103]:
%%timeit 
res = target_mean_v9(data, 'y', 'x')

100 loops, best of 3: 7.29 ms per loop


# target_mean_vn

使用了 c++ 的map尝试一下，测试结果不如 v9

In [96]:
%%cython --cplus 
from libcpp.map cimport map
cimport cython
import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_vn(data, y_name, x_name):
  cdef:
    int data_shape = data.shape[0]
    double[:] result = np.zeros(data_shape, dtype=np.float64)
    map[long, long] val_dict
    map[long, long] cnt_dict
    long[:] x_values = data[x_name].values
    long[:] y_values = data[y_name].values
  for i in range(data_shape):
    if val_dict.count(x_values[i]):
      val_dict[x_values[i]] += y_values[i]
      cnt_dict[x_values[i]] += 1
    else:
      val_dict[x_values[i]] = y_values[i]
      cnt_dict[x_values[i]] = 0

  for i in range(data_shape):
    result[i] = (val_dict[x_values[i]] - y_values[i]) / (cnt_dict[x_values[i]] - 1)
  return result

In [98]:
%%timeit
res = target_mean_vn(data, 'y', 'x')

1000 loops, best of 3: 392 µs per loop


In [ ]:
print(res)